<a href="https://colab.research.google.com/github/comptech-winter-school/covid19-prediction/blob/etl/prepare_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from pathlib import Path

In [ ]:
path = "COVID-19/csse_covid_19_data/csse_covid_19_daily_reports"
all_files = Path(path).glob("*.csv") 

df = pd.concat(map(pd.read_csv, all_files))

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
def remove_small_countries(df):
    countries = df.groupby("Country_Region").last().query('Confirmed > 100000').index.to_list()
    return df[df.Country_Region.isin(countries)]

In [ ]:
def report_shape(df):
    print(df.shape)
    return df

In [ ]:
def merge_columns(f1, f2, df):
    d = df.loc[:, [f1, f2]].copy()
    d.loc[d[f1].isna(), f1] = d.loc[d[f1].isna(), f2]
    return d[f1]

In [ ]:
def add_new_cases(data):

    temp_df, df_ = pd.DataFrame(), pd.DataFrame()
    for country in data['Country_Region'].unique():
        temp_df = data.loc[data['Country_Region'] == country, :].copy()
        temp_df['New_Cases'] = (temp_df['Confirmed'] - temp_df['Confirmed'].shift(1))
        df_ = pd.concat([df_, temp_df], axis=0).dropna()
    
    return df_

In [ ]:
(df
 .assign(Country_Region = merge_columns("Country_Region", "Country/Region", df))
 .assign(Last_Update = merge_columns("Last_Update", "Last Update", df))
 .assign(Last_Update = pd.to_datetime(df["Last_Update"]).dt.date)
 .loc[:, ["Country_Region", "Last_Update", "Confirmed"]]
 .dropna()
 .groupby(["Country_Region", "Last_Update"])
 .sum()
 .reset_index()
 .set_index("Last_Update")
 .sort_index()
 .pipe(remove_small_countries)
 .pipe(add_new_cases)
#  .pipe(report_shape)
 .to_parquet("tmp/data.parquet")
)